<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3 Web APIs and NLP

_Authors: Joel Quek (SG)_

# Web Scraping for r/investing

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime

import random

import json
import csv

In [3]:
url = 'https://api.pushshift.io/reddit/search/submission'

# Function to Pull 5000 rows of data

## Current Unix Timestamp [[Epoch Converter]](https://www.epochconverter.com/)

In [4]:
presentDate = datetime.now()
unix_timestamp = datetime.timestamp(presentDate)
print(unix_timestamp)

1667315358.335274


https://www.epochconverter.com/

## Function 

In [5]:
def PushShift5000(sub, size, present):
    url = 'https://api.pushshift.io/reddit/search/submission'
    #-----------------------------------------------------------------
    params ={
        'subreddit': str(sub),
        'size': int(size),
        'before': int(present)
    }
    res = requests.get(url,params)
    data=res.json()
    posts = data['data']
    df=pd.DataFrame(posts)
    max_size = df.shape[0]-1
    #------------------------------------------------------------------
    while df.shape[0] < size:
        params2 ={
            'subreddit': str(sub),
            'size': int(size),
            'before': posts[max_size]['created_utc']
        }
        res2 = requests.get(url,params2)
        data2=res2.json()
        posts = data2['data']
        df2=pd.DataFrame(posts) 
        df=pd.concat([df,df2],ignore_index=True)
    return df

# r/investing

In [6]:
investing = PushShift5000('investing', 3600, unix_timestamp)

In [7]:
investing.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,preview,suggested_sort,author_flair_background_color,author_flair_text_color,banned_by,author_cakeday,link_flair_template_id,link_flair_text,call_to_action,category
0,[],False,Feisty-Astronaut5398,None,[],None,text,t2_c5nwrc40,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Slapdaddy13,None,[],None,text,t2_iy51bdtq,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,catsRawesome123,None,[],None,text,t2_z72rp,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,hydrocyanide,None,[],None,text,t2_ejq52,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,KyloRenSucks,None,[],None,text,t2_4ydl1bh9,False,False,...,"{'enabled': False, 'images': [{'id': 'mLyIJBTi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
investing.shape[0]

3750

In [9]:
investing.iloc[investing.shape[0]-1]['created_utc']

1662574715

In [10]:
investing2 = PushShift5000('investing', 1250, investing.iloc[investing.shape[0]-1]['created_utc'])

In [11]:
investing2.shape[0]

1498

In [12]:
investing=pd.concat([investing,investing2],ignore_index=True)
investing.shape

(5248, 71)

In [13]:
investing[['subreddit', 'author', 'selftext', 'title']].head(20)

,subreddit,author,selftext,title
0,investing,Feisty-Astronaut5398,[removed],Looking for a mutual fund tracker that automat...
1,investing,Slapdaddy13,[removed],Doge Coin pump or dump til the cows come home?
2,investing,catsRawesome123,[removed],What are benefits of buying t-bill through Tre...
3,investing,hydrocyanide,https://www.treasurydirect.gov/savings-bonds/i...,New Series I savings bond rate is 6.89% (0.40%...
4,investing,KyloRenSucks,[removed],Treasury announces new Series I bond rate of 6...
5,investing,Electronic_Ad_701,[removed],Investment question? New to this and 19 I’m eager
6,investing,Quiet-Literature2251,"Hey, this is my current plan. I'm in sales and...",Making substantial money now but not sure what...
7,investing,4619,[removed],What happens in an emerging markets ETF when a...
8,investing,MillerEmma90,[removed],"sFOX Review: Features, Pros, Cons &amp; Best s..."
9,investing,Background_Stock1212,In Morgan Stanley's 2019 The Recession Playboo...,History suggests that the Japanese yen will st...


In [14]:
investing.to_csv('datasets/investing.csv')

In [15]:
investing.iloc[investing.shape[0]-1]['created_utc']

1660670168

In [34]:
datetime.now(int(1660670168))

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'int'

# r/stockmarket

In [16]:
stockmarket = PushShift5000('StockMarket', 2500, unix_timestamp)

In [17]:
stockmarket.shape

(2500, 80)

In [18]:
stockmarket.iloc[2499]['created_utc']

1664389704

In [19]:
stockmarket2 = PushShift5000('StockMarket', 2500, stockmarket.iloc[stockmarket.shape[0]-1]['created_utc'])

In [20]:
stockmarket=pd.concat([stockmarket,stockmarket2],ignore_index=True)
stockmarket.shape

(5245, 80)

In [21]:
stockmarket.iloc[0]

all_awardings                         []
allow_live_comments                False
author                           Timkain
author_flair_css_class              None
author_flair_richtext                 []
                                  ...   
is_gallery                           NaN
author_flair_background_color        NaN
author_flair_text_color              NaN
author_cakeday                       NaN
banned_by                            NaN
Name: 0, Length: 80, dtype: object

In [22]:
stockmarket.iloc[2503]

all_awardings                          []
allow_live_comments                 False
author                           GeArci21
author_flair_css_class               None
author_flair_richtext                  []
                                   ...   
is_gallery                            NaN
author_flair_background_color         NaN
author_flair_text_color               NaN
author_cakeday                        NaN
banned_by                             NaN
Name: 2503, Length: 80, dtype: object

In [23]:
stockmarket.to_csv('datasets/stockmarket.csv')

In [24]:
stockmarket.iloc[stockmarket.shape[0]-1]['created_utc']

1660732530

# r/wallstreetbets

In [25]:
wallstreetbets = PushShift5000('wallstreetbets', 500, unix_timestamp)

In [26]:
pd.set_option('display.max_columns', None)

wallstreetbets.head()

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_created_from_ads_ui,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,author_flair_template_id,removed_by_category,media_metadata,gallery_data,is_gallery,live_audio,poll_data,media,media_embed,secure_media,secure_media_embed,author_cakeday
0,[],False,DeadLightsOut,,None,[],None,,text,t2_nxg5b16,False,False,False,[],False,False,1667315341,i.redd.it,https://www.reddit.com/r/wallstreetbets/commen...,{},yjb4bs,False,True,False,False,True,True,False,False,#014980,meme,"[{'e': 'text', 't': 'Meme'}]",0513bea8-4f64-11e9-886d-0e2b4fe7300c,Meme,light,richtext,False,False,True,0,0,False,some_ads,/r/wallstreetbets/comments/yjb4bs/the_turkey_p...,False,image,"{'enabled': True, 'images': [{'id': '-BZK2Iq97...",7,1667315352,1,,True,False,False,wallstreetbets,t5_2th52,12979302,public,confidence,https://b.thumbs.redditmedia.com/zkDy2eT1uQNf3...,105.0,140.0,The Turkey Pivot Narrative Has Begun!,0,[],1.0,https://i.redd.it/45tz74a0vcx91.jpg,https://i.redd.it/45tz74a0vcx91.jpg,some_ads,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,catbulliesdog,#ffdf00,None,"[{'e': 'text', 't': 'Is long on agriculture fu...",Is long on agriculture futes,dark,richtext,t2_a57oe8f9,False,False,True,[],False,False,1667315194,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},yjb26w,False,False,False,False,False,False,True,False,#ff4500,chart,"[{'e': 'text', 't': 'Chart'}]",e7bce9ea-4b49-11eb-ad96-0e210774c543,Chart,light,richtext,False,False,True,1,0,False,some_ads,/r/wallstreetbets/comments/yjb26w/fall_harvest...,False,NaN,NaN,7,1667315205,1,[removed],True,False,False,wallstreetbets,t5_2th52,12979280,public,confidence,https://a.thumbs.redditmedia.com/RU3jks-LpgP8Z...,140.0,140.0,"Fall Harvest info coming in, generally speakin...",0,[],1.0,https://www.reddit.com/r/wallstreetbets/commen...,NaN,some_ads,7,5b951270-9b43-11e8-8a57-0e532bbfe7e8,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,kidicaru59,#ffffff,None,"[{'e': 'text', 't': 'first tard'}]",first tard,dark,richtext,t2_5xucsg1,False,False,False,[],False,False,1667315114,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},yjb10g,False,True,False,False,False,True,True,False,#349e48,profit,"[{'e': 'text', 't': 'Gain'}]",6bd9394c-0a95-11e9-82db-0eb3a11267a0,Gain,light,richtext,False,False,False,0,0,False,some_ads,/r/wallstreetbets/comments/yjb10g/kweb_calls_1...,False,NaN,NaN,7,1667315125,1,"I needed the cash and decided to take profits,...",True,False,False,wallstreetbets,t5_2th52,12979276,public,confidence,https://b.thumbs.redditmedia.com/O5yGZy7QVJqIY...,71.0,140.0,KWEB Calls - $10k gain - KWEB still cheap and ...,0,[],1.0,https://www.reddit.com/r/wallstreetbets/commen...,NaN,some_ads,7,acf21f8a-b20b-11e8-8717-0ee9cf2d415a,NaN,"{'0f5op17ztcx91': {'e': 'Image', 'id': '0f5op1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Aepecks,NaN,None,[],None,NaN,text,t2_tifwa051,False,False,False,[],False,False,1667315067,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},yjb0b6,False,True,False,False,False,True,True

In [27]:
wallstreetbets2 = PushShift5000('wallstreetbets', 500, wallstreetbets.iloc[wallstreetbets.shape[0]-1]['created_utc'])

In [28]:
wallstreetbets=pd.concat([wallstreetbets,wallstreetbets2],ignore_index=True)
wallstreetbets.shape

(1249, 82)

In [29]:
wallstreetbets.shape[0]-1

1248

In [30]:
wallstreetbets.to_csv('datasets/wallstreetbets.csv')

In [31]:
# wallstreetbets2 = PushShift5000('wallstreetbets', 500, wallstreetbets.iloc[wallstreetbets.shape[0]-1]['created_utc'])